# Определение рыночной стоимости поддержанных автомобилей

Сервис по продаже поддержанных автомобилей разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Описание данных

**Признаки:**
+ DateCrawled — дата скачивания анкеты из базы
+ VehicleType — тип автомобильного кузова
+ RegistrationYear — год регистрации автомобиля (год выпуска)
+ Gearbox — тип коробки передач
+ Power — мощность (л. с.)
+ Model — модель автомобиля
+ Kilometer — пробег (км)
+ RegistrationMonth — месяц регистрации автомобиля
+ FuelType — тип топлива
+ Brand — марка автомобиля
+ NotRepaired — была машина в ремонте или нет
+ DateCreated — дата создания анкеты
+ NumberOfPictures — количество фотографий автомобиля
+ PostalCode — почтовый индекс владельца анкеты (пользователя)
+ LastSeen — дата последней активности пользователя

**Целевой признак:**
+ Price — цена (евро)

# 1. Подготовка данных

In [3]:
import pandas as pd
import lightgbm as lgb
import numpy as np

from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

In [4]:
data = pd.read_csv('/datasets/autos.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


##### Устранение дубликатов

In [5]:
data.duplicated().sum()

4

In [6]:
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

##### Устранение выбивающихся и неадекватных значений

In [7]:
data.loc[data['RegistrationYear'] > 2016, 'RegistrationYear'] = np.nan
data.loc[data['RegistrationYear'] < 1940, 'RegistrationYear'] = np.nan

data.loc[data['Price'] == 0, 'Price'] = np.nan

data.loc[data['Power'] == 0, 'Power'] = np.nan
q1 = data['Power'].quantile(.25)
q3 = data['Power'].quantile(.75)
data.loc[data['Power'] > (q3 + 3 * (q3-q1)), 'Power'] = np.nan

In [8]:
features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired', 'PostalCode']
for feature in features:
    data[feature].fillna(value='None', inplace=True)

##### Устранение пропусков

In [9]:
#data.dropna(subset=['Price'], inplace=True)
data.dropna(subset=['Price', 'Power', 'RegistrationYear'], inplace=True)

##### Устранение ненужных столбцов

In [10]:
data.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'LastSeen'],axis=1, inplace=True)

##### Разделение на выборки

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
        data.drop('Price', axis=1), data['Price'], test_size=0.25, random_state=42)

##### One-hot-encoding для линейной регрессии

In [12]:
data_ohe = pd.get_dummies(data, drop_first=True)
f_train_ohe, f_test_ohe, t_train_ohe, t_test_ohe = train_test_split(
        data_ohe.drop('Price', axis=1), data_ohe['Price'], test_size=0.25, random_state=42)

# 2. Обучение моделей

##### Линейная регрессия

In [11]:
%%time

model_lr = LinearRegression()
model_lr.fit(f_train_ohe, t_train_ohe)

CPU times: user 19.6 s, sys: 5.15 s, total: 24.7 s
Wall time: 24.7 s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

##### Случайный лес

In [ ]:
model_rfr = RandomForestRegressor(random_state=42)

grid_param = {'n_estimators' : [50 * x for x in range(1, 6)],
              'max_depth' : [5 * x for x in range(1, 6)]}

grid_search = GridSearchCV(model_rfr, grid_param, scoring='neg_mean_squared_error', n_jobs=-1, cv=3)
grid_search.fit(f_train_ohe, t_train_ohe)
grid_search.best_params_

{'max_depth': 25, 'n_estimators': 250}

In [12]:
%%time

model_rfr = RandomForestRegressor(n_estimators=250 , max_depth=25 , random_state=42)
model_rfr.fit(f_train_ohe, t_train_ohe)

CPU times: user 24min 43s, sys: 2.6 s, total: 24min 45s
Wall time: 25min 10s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=250,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

##### LightGBM

In [13]:
model_lgbmr = lgb.LGBMRegressor(random_state=42)

cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired', 'PostalCode']

for column in cat_features:
    features_train[column] = features_train[column].astype('category')
    features_test[column] = features_test[column].astype('category')

In [ ]:
grid_param = {
    'max_depth': [10, 20, 30],
    'num_leaves': [30, 50, 70],
    'learning_rate': [0.03],
    'num_boost_round': [300, 400, 500, 600]
}

grid_search = GridSearchCV(model_lgbmr, grid_param, scoring='neg_mean_squared_error', n_jobs=-1, cv=3, verbose=10)
grid_search.fit(features_train, target_train)
grid_search.best_params_

{'learning_rate': 0.03,
 'max_depth': 20,
 'num_boost_round': 600,
 'num_leaves': 70}

In [14]:
%%time

data_train = lgb.Dataset(features_train, label=target_train)

params =  {
    'max_depth': 20,
    'num_leaves': 70,
    'learning_rate': 0.03,
    'num_boost_round': 600
}


model_lgbmr =lgb.train(params, data_train)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


CPU times: user 2min 55s, sys: 617 ms, total: 2min 55s
Wall time: 2min 58s


# 3. Анализ моделей

##### Линейная регрессия

In [15]:
%%time

pred_test = model_lr.predict(f_test_ohe)
mean_squared_error(t_test_ohe, pred_test) ** 0.5

CPU times: user 84 ms, sys: 60.4 ms, total: 144 ms
Wall time: 177 ms


2565.826057264036

##### Случайный лес

In [16]:
%%time

pred_test = model_rfr.predict(f_test_ohe)
mean_squared_error(t_test_ohe, pred_test) ** 0.5

CPU times: user 9.22 s, sys: 72.2 ms, total: 9.29 s
Wall time: 9.42 s


1522.9202003431492

##### LightGBM

In [17]:
%%time

pred_test = model_lgbmr.predict(features_test)
mean_squared_error(target_test, pred_test) ** 0.5

CPU times: user 19.9 s, sys: 27.4 ms, total: 19.9 s
Wall time: 20 s


1529.2100050276556

##### Константная модель

In [14]:
pred_median = np.ones(target_test.shape) * target_train.median()
mean_squared_error(target_test, pred_median) ** 0.5

4878.672630728247

### Итоги

+ Все модели показали себя лучше константной модели
+ Самый быстрый, но самый грубый результат даёт модель линейной регрессии
+ Наилучший показатель метрики RMSE показала модель случайного леса (1522.9), однако модель очень долго обучается (25 минут)
+ Модель LightGBM обучается и даёт предсказание за вменяемое время, RMSE=1529.2 : чуть хуже, чем у случайного леса

P.S. В LightGBM наверняка можно подобрать гиперпараметры, которые дадут лучший результат по RMSE, однако grid search слишком долго выполняется.